In [1]:
# 모듈 불러오기
import pandas as pd
import numpy as np
import time
import re
import json
import folium
import googlemaps

from tqdm import tqdm_notebook
from folium.features import DivIcon
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By

In [62]:
# 홈페이지를 selenium으로 열기
driver = webdriver.Chrome()
driver.get('https://www.justwatch.com/kr/%EB%8F%99%EC%98%81%EC%83%81%EC%84%9C%EB%B9%84%EC%8A%A4/netflix/TV-%ED%94%84%EB%A1%9C%EA%B7%B8%EB%9E%A8?genres=drm&release_year_from=2020&release_year_until=2023&sort_by=release_year')

In [73]:
# 현재 페이지 내의 정보를 저장
raw = driver.page_source
raw

'<html dir="ltr" lang="ko" class="plt-desktop md hydrated" mode="md"><head><meta charset="utf-8"><style type="text/css">:root, :host {\n  --fa-font-solid: normal 900 1em/1 "Font Awesome 6 Solid";\n  --fa-font-regular: normal 400 1em/1 "Font Awesome 6 Regular";\n  --fa-font-light: normal 300 1em/1 "Font Awesome 6 Light";\n  --fa-font-thin: normal 100 1em/1 "Font Awesome 6 Thin";\n  --fa-font-duotone: normal 900 1em/1 "Font Awesome 6 Duotone";\n  --fa-font-sharp-solid: normal 900 1em/1 "Font Awesome 6 Sharp";\n  --fa-font-sharp-regular: normal 400 1em/1 "Font Awesome 6 Sharp";\n  --fa-font-sharp-light: normal 300 1em/1 "Font Awesome 6 Sharp";\n  --fa-font-brands: normal 400 1em/1 "Font Awesome 6 Brands";\n}\n\nsvg:not(:root).svg-inline--fa, svg:not(:host).svg-inline--fa {\n  overflow: visible;\n  box-sizing: content-box;\n}\n\n.svg-inline--fa {\n  display: var(--fa-display, inline-block);\n  height: 1em;\n  overflow: visible;\n  vertical-align: -0.125em;\n}\n.svg-inline--fa.fa-2xs {\n  v

In [74]:
# BeautifulSoup을 이용하여 html 파싱
soup = BeautifulSoup(raw, 'html.parser')

In [ ]:
#base > div.title-list.title-list--CLS-block > div:nth-child(2) > div > div > div:nth-child(1) > div > div:nth-child(2) > a
#base > div.title-list.title-list--CLS-block > div:nth-child(2) > div > div > div:nth-child(1) > div > div:nth-child(3) > a

In [75]:
# 개수 확인
len(soup.select('.title-list-grid')[0])

# 2부터 723까지 존재함 -> 즉 722개 (화면 상 데이터 개수와 일치)
# nth-child(1)은 없는 데이터.

723

In [94]:
# url_list 만들기
base_url = 'https://www.justwatch.com'
url_list = []

for i in tqdm_notebook(range(2, len(soup.select('.title-list-grid')[0])+1)):
    tmp = soup.select_one('#base > div.title-list.title-list--CLS-block > div:nth-child(2) > div > div > div:nth-child(1) > div > div:nth-child('+str(i)+') > a')
    tmp_url = tmp.get('href')
    url = base_url + tmp_url
    url_list.append(url)

C:\Users\mycom\AppData\Local\Temp\ipykernel_40644\2962605383.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(2, len(soup.select('.title-list-grid')[0])+1)):


  0%|          | 0/722 [00:00<?, ?it/s]

In [95]:
url_list

['https://www.justwatch.com/kr/TV-프로그램/analrogeu-seukweodeu',
 'https://www.justwatch.com/kr/TV-프로그램/wave-makers',
 'https://www.justwatch.com/kr/TV-프로그램/yu-shou-xi-jia-yan-shang-suru',
 'https://www.justwatch.com/kr/TV-프로그램/moraeedo-ggoci-pinda',
 'https://www.justwatch.com/kr/TV-프로그램/captain-laserhawk-a-blood-dragon-remix',
 'https://www.justwatch.com/kr/TV-프로그램/yuyubaegseo',
 'https://www.justwatch.com/kr/TV-프로그램/sky-high',
 'https://www.justwatch.com/kr/TV-프로그램/nabbeuneomma',
 'https://www.justwatch.com/kr/TV-프로그램/se-gaeyi-bimil',
 'https://www.justwatch.com/kr/TV-프로그램/sleeping-dog',
 'https://www.justwatch.com/kr/TV-프로그램/el-cuerpo-en-llamas',
 'https://www.justwatch.com/kr/TV-프로그램/keuriceo',
 'https://www.justwatch.com/kr/TV-프로그램/guns-and-gulaabs',
 'https://www.justwatch.com/kr/TV-프로그램/puba',
 'https://www.justwatch.com/kr/TV-프로그램/barracuda-queens',
 'https://www.justwatch.com/kr/TV-프로그램/the-diplomat',
 'https://www.justwatch.com/kr/TV-프로그램/vortex',
 'https://www.justwatch.com/kr

In [96]:
driver.close()

In [97]:
# 드라마 정보 크롤링 테스트
driver = webdriver.Chrome()
driver.get(url_list[0])

In [99]:
# 현재 페이지 내의 정보를 저장
raw = driver.page_source

# BeautifulSoup을 이용하여 html 파싱
soup = BeautifulSoup(raw, 'html.parser')

In [144]:
# 테스트
movie_list = []; movie_data = {}

for x in url_list[:5]:
    driver = webdriver.Chrome()
    driver.get(x)
    raw = driver.page_source
    soup = BeautifulSoup(raw, 'html.parser')

    k_name = soup.select_one('#base > div:nth-child(1) > div.jw-info-box > div > div.jw-info-box__container-content > div:nth-child(2) > div.title-block__container > div.title-block > div > h1').text[1:-1]
    f_name = soup.select_one('#base > div:nth-child(1) > div.jw-info-box > div > div.jw-info-box__container-content > div:nth-child(2) > div.title-block__container > div.title-block > h3').text[5:-1]
    season = soup.select_one('#base > div:nth-child(1) > div.jw-info-box > div > div.jw-info-box__container-content > div:nth-child(2) > div:nth-child(2) > h2').text[1:-1]
    genre = soup.select_one('#base > div:nth-child(1) > div.jw-info-box > div > div.jw-info-box__container-sidebar > div > aside > div.hidden-sm.visible-md.visible-lg.title-sidebar > div.title-info.title-info > div:nth-child(3) > div').text
    time = soup.select_one('#base > div:nth-child(1) > div.jw-info-box > div > div.jw-info-box__container-sidebar > div > aside > div.hidden-sm.visible-md.visible-lg.title-sidebar > div.title-info.title-info > div:nth-child(4) > div').text
    limit = soup.select_one('#base > div:nth-child(1) > div.jw-info-box > div > div.jw-info-box__container-sidebar > div > aside > div.hidden-sm.visible-md.visible-lg.title-sidebar > div.title-info.title-info > div:nth-child(5) > div').text
    country = soup.select_one('#base > div:nth-child(1) > div.jw-info-box > div > div.jw-info-box__container-sidebar > div > aside > div.hidden-sm.visible-md.visible-lg.title-sidebar > div.title-info.title-info > div:nth-child(6) > div').text
    director = soup.select_one('#base > div:nth-child(1) > div.jw-info-box > div > div.jw-info-box__container-sidebar > div > aside > div.hidden-sm.visible-md.visible-lg.title-sidebar > div.title-info.title-info > div:nth-child(7) > div > span > span').text[1:-1]
    story = soup.select_one('#base > div:nth-child(1) > div.jw-info-box > div > div.jw-info-box__container-content > div:nth-child(2) > div:nth-child(6) > p').text[1:-1]

    movie_data = {
        'k_name': k_name, 'f_name': f_name, 'season': season,
        'genre': genre, 'time': time,
        'limit': limit, 'country': country, 'director': director, 'story': story
    }
    
    movie_list.append(movie_data)

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
# 

In [105]:
# 한글 이름
soup.select_one('#base > div:nth-child(1) > div.jw-info-box > div > div.jw-info-box__container-content > div:nth-child(2) > div.title-block__container > div.title-block > div > h1').text[1:-1]

'아날로그 스쿼드'

In [107]:
# 원제
soup.select_one('#base > div:nth-child(1) > div.jw-info-box > div > div.jw-info-box__container-content > div:nth-child(2) > div.title-block__container > div.title-block > h3').text[5:-1]

'Analog Squad ทีมรักนักหลอก'

In [109]:
# 시즌 수
soup.select_one('#base > div:nth-child(1) > div.jw-info-box > div > div.jw-info-box__container-content > div:nth-child(2) > div:nth-child(2) > h2').text[1:-1]

'1개 시즌'

In [100]:
# 장르
soup.select_one('#base > div:nth-child(1) > div.jw-info-box > div > div.jw-info-box__container-sidebar > div > aside > div.hidden-sm.visible-md.visible-lg.title-sidebar > div.title-info.title-info > div:nth-child(3) > div').text

'드라마'

In [110]:
# 재생 시간
soup.select_one('#base > div:nth-child(1) > div.jw-info-box > div > div.jw-info-box__container-sidebar > div > aside > div.hidden-sm.visible-md.visible-lg.title-sidebar > div.title-info.title-info > div:nth-child(4) > div').text

'1시간 0분'

In [112]:
# 연령 등급
soup.select_one('#base > div:nth-child(1) > div.jw-info-box > div > div.jw-info-box__container-sidebar > div > aside > div.hidden-sm.visible-md.visible-lg.title-sidebar > div.title-info.title-info > div:nth-child(5) > div').text

'15'

In [111]:
# 제조 국가
soup.select_one('#base > div:nth-child(1) > div.jw-info-box > div > div.jw-info-box__container-sidebar > div > aside > div.hidden-sm.visible-md.visible-lg.title-sidebar > div.title-info.title-info > div:nth-child(6) > div').text

'태국'

In [114]:
# 감독
soup.select_one('#base > div:nth-child(1) > div.jw-info-box > div > div.jw-info-box__container-sidebar > div > aside > div.hidden-sm.visible-md.visible-lg.title-sidebar > div.title-info.title-info > div:nth-child(7) > div > span > span').text[1:-1]

'Nithiwat Tharathorn'

In [138]:
# 배우
soup.select('#base > div:nth-child(1) > div.jw-info-box > div > div.jw-info-box__container-content > div:nth-child(7) > div:nth-child(1) > div:nth-child(2) > div.hidden-horizontal-scrollbar.title-credits')[0]

<div class="hidden-horizontal-scrollbar title-credits" horizontal-scroll-style="basic" item-amount="18"><div class="hidden-horizontal-scrollbar__items"><div class="title-credits__actor"><span class="title-credit-name" data-v-2346c4b4=""> Nopachai Jayanama </span><div class="title-credits__actor--role"><div class="title-credits__actor--role--name"><strong title="Pond">Pond</strong></div></div></div><div class="title-credits__actor"><span class="title-credit-name" data-v-2346c4b4=""> Kullanat Preeyawat </span><div class="title-credits__actor--role"><div class="title-credits__actor--role--name"><strong title="Lilly / Mam">Lilly / Mam</strong></div></div></div><div class="title-credits__actor"><span class="title-credit-name" data-v-2346c4b4=""> Kritsanapoom Pibulsonggram </span><div class="title-credits__actor--role"><div class="title-credits__actor--role--name"><strong title="Keg / Mon">Keg / Mon</strong></div></div></div><div class="title-credits__actor"><span class="title-credit-name" d

In [142]:
soup.select('#base > div:nth-child(1) > div.jw-info-box > div > div.jw-info-box__container-content > div:nth-child(7) > div:nth-child(1) > div:nth-child(2) > div.hidden-horizontal-scrollbar.title-credits > div > div:nth-child(1) > span')

[<span class="title-credit-name" data-v-2346c4b4=""> Nopachai Jayanama </span>]

In [141]:
# 시놉시스
soup.select_one('#base > div:nth-child(1) > div.jw-info-box > div > div.jw-info-box__container-content > div:nth-child(2) > div:nth-child(6) > p').text[1:-1]

'소원하게 지내던 아버지가 위중하다는 소식을 들은 중년의 남자. 마지막으로 아버지를 만나러 가기 위해 가짜 가족을 만들 계획을 꾸민다.'